# Silero VAD + webrtcvad

In [22]:
!pip3 install silero-vad # pyAudioAnalysis # webrtcvad # torch torchvision torchaudio

  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [28]:
import torch
import numpy as np
import webrtcvad
import soundfile as sf
from scipy.signal import butter, lfilter 
import scipy.fft
import librosa

from silero_vad import load_silero_vad, read_audio, get_speech_timestamps

In [33]:
# Загрузка модели Silero VAD
model = load_silero_vad()

def remove_speech_silero_vad(audio_file):
    # Чтение аудио
    audio, sample_rate = sf.read(audio_file)
    audio = librosa.to_mono(audio)
    # Использование Silero VAD
    speech_timestamps = get_speech_timestamps(audio, model)  # Return speech timestamps in seconds (default is samples))
    print(speech_timestamps)
    # Вырезаем участки с речью
    speech_free_audio = []
    for start, stop in speech_timestamps:
        speech_free_audio.append(audio[start:stop])

    speech_free_audio = np.concatenate(speech_free_audio, axis=0)
    return speech_free_audio, sample_rate

def remove_speech_webrtcvad(audio_file, sample_rate):
    # Инициализация webrtcvad
    vad = webrtcvad.Vad(1)  # Порог чувствительности 1 (низкий)
    audio, sample_rate = sf.read(audio_file)
    frame_duration = 0.02  # 20 ms
    frame_size = int(sample_rate * frame_duration)
    frames = [audio[i:i+frame_size] for i in range(0, len(audio), frame_size)]
    speech_free_audio = []

    for frame in frames:
        if vad.is_speech(frame.tobytes(), sample_rate):
            continue  # Пропускаем фрейм с речью
        speech_free_audio.append(frame)

    return np.concatenate(speech_free_audio, axis=0)


In [34]:

# Основной пайплайн
audio_file = "../data/норма/Соболева Ирина Анатольевна.wav"
speech_free_audio, sample_rate = remove_speech_silero_vad(audio_file)
clean_audio = remove_speech_webrtcvad(audio_file, sample_rate)

# Сохранение очищенного аудио
sf.write("../data/норма/Соболева Ирина Анатольевна_cleaned.wav", clean_audio, sample_rate)



[]


ValueError: need at least one array to concatenate

# Spleeter или Meta Voice Separation

In [6]:
!pip install tensorflow==2.7.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.7.0 (from versions: 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.7.0


In [41]:
from spleeter.separator import Separator
import soundfile as sf

# Инициализация Spleeter для разделения звуков на два источника (вокал и аккомпанемент)
separator = Separator('spleeter:2stems')

# Разделение аудио
separator.separate_to_file('../data/норма/Соболева Ирина Анатольевна.wav', './')

# Загрузка чистых звуков (например, аккомпанемент)
clean_audio, sample_rate = sf.read('./Соболева Ирина Анатольевна.wav')

# Сохранение очищенного аудио
sf.write('cleaned_auscultation_no_speech.wav', clean_audio, sample_rate)


ModuleNotFoundError: No module named 'spleeter'

# Source separation

In [8]:
!pip install openunmix

In [31]:
checkpoint.keys()

odict_keys(['input_mean', 'input_scale', 'output_scale', 'output_mean', 'sample_rate', 'stft.window', 'transform.0.window', 'fc1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'lstm.weight_ih_l0', 'lstm.weight_hh_l0', 'lstm.bias_ih_l0', 'lstm.bias_hh_l0', 'lstm.weight_ih_l0_reverse', 'lstm.weight_hh_l0_reverse', 'lstm.bias_ih_l0_reverse', 'lstm.bias_hh_l0_reverse', 'lstm.weight_ih_l1', 'lstm.weight_hh_l1', 'lstm.bias_ih_l1', 'lstm.bias_hh_l1', 'lstm.weight_ih_l1_reverse', 'lstm.weight_hh_l1_reverse', 'lstm.bias_ih_l1_reverse', 'lstm.bias_hh_l1_reverse', 'lstm.weight_ih_l2', 'lstm.weight_hh_l2', 'lstm.bias_ih_l2', 'lstm.bias_hh_l2', 'lstm.weight_ih_l2_reverse', 'lstm.weight_hh_l2_reverse', 'lstm.bias_ih_l2_reverse', 'lstm.bias_hh_l2_reverse', 'fc2.weight', 'bn2.weight', 'bn2.bias', 'bn2.running_mean', 'bn2.running_var', 'bn2.num_batches_tracked', 'fc3.weight', 'bn3.weight', 'bn3.bias', 'bn3.running_mean', 'bn3.running_var', 'bn3.num_

In [39]:
import torch
import torchaudio
import soundfile as sf
import os
from openunmix.model import OpenUnmix
import numpy as np
import librosa

# === ПАРАМЕТРЫ ===
AUDIO_PATH = "../data/норма/Соболева Ирина Анатольевна.wav"
MODEL_PATH = "/Users/annatekuceva/.cache/torch/hub/checkpoints/vocals-bccbd9aa.pth"
OUTPUT_PATH = "background_only.wav"

# === ЗАГРУЗКА АУДИО ===
# OpenUnmix работает на 44.1 kHz, stereo
waveform, sr = librosa.load(AUDIO_PATH, sr=44100, mono=False)
if waveform.ndim == 1:
    waveform = np.expand_dims(waveform, axis=0)  # из mono в [1, n]
elif waveform.shape[0] != 2:
    raise ValueError("Audio must be stereo (2 channels)")

waveform = torch.tensor(waveform, dtype=torch.float32)

# === ЗАГРУЗКА МОДЕЛИ ИЗ ВЕСОВ ===
checkpoint = torch.load(MODEL_PATH, map_location="cpu")
filtered_state_dict = {k: v for k, v in checkpoint.items() if not any(
    skip in k for skip in ["sample_rate", "stft.", "transform.0."])}

umx = OpenUnmix()

umx.load_state_dict(filtered_state_dict)
umx.eval()



RuntimeError: Error(s) in loading state_dict for OpenUnmix:
	size mismatch for input_mean: copying a param with shape torch.Size([1487]) from checkpoint, the shape in current model is torch.Size([4096]).
	size mismatch for input_scale: copying a param with shape torch.Size([1487]) from checkpoint, the shape in current model is torch.Size([4096]).
	size mismatch for output_scale: copying a param with shape torch.Size([2049]) from checkpoint, the shape in current model is torch.Size([4096]).
	size mismatch for output_mean: copying a param with shape torch.Size([2049]) from checkpoint, the shape in current model is torch.Size([4096]).
	size mismatch for fc1.weight: copying a param with shape torch.Size([1024, 2974]) from checkpoint, the shape in current model is torch.Size([512, 8192]).
	size mismatch for bn1.weight: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for bn1.bias: copying a param with shape torch.

In [ ]:

# === STFT ===
n_fft = 4096
hop_length = 1024
win_length = 4096
window = torch.hann_window(win_length)

# Преобразуем в частотную область
X = torch.stft(waveform, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
               window=window, return_complex=True)

# === ПРЯМОЕ ПРИМЕНЕНИЕ МОДЕЛИ ===
with torch.no_grad():
    X_mag = X.abs().permute(1, 0, 2)  # (freq, channel, time) -> (time, channel, freq)
    X_mag = X_mag.numpy()
    estimates = umx(torch.tensor(X_mag)).detach().numpy()  # [T, C, F]

# Восстанавливаем фазу и обратно в time-domain
estimates = torch.tensor(estimates).permute(1, 2, 0)  # -> [C, F, T]
estimates = torch.view_as_complex(estimates)  # [C, F, T] -> complex
background_stft = estimates
background_audio = torch.istft(background_stft, n_fft=n_fft, hop_length=hop_length,
                               win_length=win_length, window=window, length=waveform.shape[1])

# === СОХРАНЕНИЕ ===
background_audio_np = background_audio.numpy()
sf.write(OUTPUT_PATH, background_audio_np.T, sr)

print(f"Сохранено безречевое аудио в: {OUTPUT_PATH}")

# wavelet \ noisereduce \ banda stop filter

In [44]:
!pip install noisereduce

In [45]:
import librosa
import soundfile as sf
import noisereduce as nr
import numpy as np
import pywt

/Users/annatekuceva/PycharmProjects/stetoscope/ElectronicStetoscope-Automizer-/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
def load_audio(path):
    """Загрузка аудиофайла"""
    audio, sr = librosa.load(path, sr=None, mono=True)
    return audio, sr

def remove_voice(audio, sr, voice_freq_range=(85, 255)):
    """Удаление голоса (основной диапазон частот человеческого голоса)"""
    # Создаем фильтр Банда-стоп для диапазона голоса
    nyquist = sr / 2.0
    low = voice_freq_range[0] / nyquist
    high = voice_freq_range[1] / nyquist
    
    # Применяем фильтр
    filtered = librosa.effects.preemphasis(audio)
    filtered = librosa.decompose.nn_filter(
        filtered,
        aggregate=np.median,
        metric='cosine',
        width=int(librosa.time_to_frames(0.1, sr=sr)))
    
    return filtered

def wavelet_denoise(audio, wavelet='db4', level=1):
    """Вейвлет-дениоинг для удаления резких шумов"""
    coeff = pywt.wavedec(audio, wavelet, mode="per")
    sigma = (1/0.6745) * np.median(np.abs(coeff[-level] - np.median(coeff[-level])))
    uthresh = sigma * np.sqrt(2 * np.log(len(audio)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode="soft") for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode="per")

def reduce_noise(audio, sr, stationary=False):
    """Подавление шумов с помощью noisereduce"""
    return nr.reduce_noise(
        y=audio,
        sr=sr,
        stationary=stationary,
        prop_decrease=1.0,
        n_fft=1024,
        win_length=512
    )

def process_heart_sound(input_path, output_path):
    """Основной процесс обработки"""
    print('1. Загрузка аудио')
    audio, sr = load_audio(input_path)
    
    print('2. Удаление голоса')
    audio = remove_voice(audio, sr)
    
    print('3. Подавление шумов')
    audio = reduce_noise(audio, sr, stationary=True)
    
    print('4. Вейвлет-дениоинг')
    audio = wavelet_denoise(audio)
    
    print('5. Нормализация')
    audio = librosa.util.normalize(audio)
    
    print('6. Сохранение результата')
    sf.write(output_path, audio, sr, subtype='PCM_16')


In [56]:
import librosa
import soundfile as sf
import noisereduce as nr
import numpy as np
from scipy import signal

def load_audio(path, target_sr=22050):
    """Загрузка с понижением частоты дискретизации для ускорения"""
    audio, sr = librosa.load(path, sr=target_sr, mono=True)
    return audio, sr

def remove_voice_fast(audio, sr, voice_range=(85, 255)):
    """Быстрый банда-стоп фильтр вместо nn_filter"""
    nyquist = sr / 2
    low = voice_range[0] / nyquist
    high = voice_range[1] / nyquist
    b, a = signal.butter(4, [low, high], btype='bandstop')
    return signal.filtfilt(b, a, audio)

def wavelet_denoise(audio, wavelet='db4', level=1):
    """Вейвлет-дениоинг для удаления резких шумов"""
    coeff = pywt.wavedec(audio, wavelet, mode="per")
    sigma = (1/0.6745) * np.median(np.abs(coeff[-level] - np.median(coeff[-level])))
    uthresh = sigma * np.sqrt(2 * np.log(len(audio)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode="soft") for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode="per")


def process_heart_sound_optimized(input_path, output_path):
    audio, sr = load_audio(input_path)
    
    # audio = wavelet_denoise(audio)

    # 1. Удаление голоса (замена nn_filter на БИХ-фильтр)
    # audio = remove_voice_fast(audio, sr)
    
    # 2. Подавление шумов с увеличенным n_fft
    audio = nr.reduce_noise(
        y=audio, 
        sr=sr,
        stationary=True,
        n_fft=2048,  # Больше разрешение для низких частот
        win_length=1024
    )
    
    # 3. Сохранение
    sf.write(output_path, audio, sr, subtype='PCM_16')

In [57]:
input_file = "../data/норма/Соболева Ирина Анатольевна.wav"  # Входной файл
output_file = "cleaned_noisereduce.wav"  # Выходной файл

process_heart_sound_optimized(input_file, output_file)
print(f"Обработка завершена. Результат сохранен в {output_file}")

Обработка завершена. Результат сохранен в cleaned_noisereduce.wav
